In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

In [2]:
# krx에서 상장기업 리스트 가져오기
stock_df = pd.read_html( 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
stock_df['종목코드'] = stock_df['종목코드'].map(lambda x: f'{x:0>6}')
corp_list = stock_df[['회사명','종목코드','업종','지역']]
corp_list

,회사명,종목코드,업종,지역
0,DSR,155660,1차 비철금속 제조업,부산광역시
1,GS,078930,기타 금융업,서울특별시
2,GS글로벌,001250,상품 종합 도매업,서울특별시
3,HDC현대산업개발,294870,건물 건설업,서울특별시
4,LG이노텍,011070,전자부품 제조업,서울특별시
...,...,...,...,...
2347,지앤이헬스케어,299480,기타 섬유제품 제조업,경기도
2348,코셋,189350,통신 및 방송 장비 제조업,광주광역시
2349,툴젠,199800,자연과학 및 공학 연구개발업,서울특별시
2350,한국미라클피플사,331660,기타 화학제품 제조업,경기도


In [3]:
corp = corp_list['회사명'].values

In [4]:
driver=webdriver.Chrome("chromedriver.exe")
driver.get('https://www.jobplanet.co.kr/search?query='+corp[1]+'&category=&_rs_con=search&_rs_act=index&_rs_element=main_search_bar')

In [5]:
# 팝업제거
icon = driver.find_element_by_class_name("tit")
icon.click()
time.sleep(0.2)

In [6]:
close = driver.find_elements_by_class_name("btn_close_x_ty1")
close[1].click()

In [7]:
# 페이지소스 가져오기
html = driver.page_source
soup = BeautifulSoup(html)

In [8]:
# 복지 및 급여
welfare = soup.find_all(class_='txt_point')[0].text
# 업무와 삶의 균형
balance = soup.find_all(class_='txt_point')[1].text
# 사내문화
culture = soup.find_all(class_='txt_point')[2].text
# 승진 기회 및 가능성
possibility = soup.find_all(class_='txt_point')[3].text
# 경영진
executive = soup.find_all(class_='txt_point')[4].text
# 총 만족도
total = soup.find_all(class_='rate_point')[0].text
name = soup.find_all(class_='word')[0].text

In [10]:
dict = {'회사명':[corp[1]],'네임':[name],'복지':[welfare],'워라벨':[balance],'사내문화':[culture],'가능성':[possibility],'경영진':[executive],'총만족도':[total]}
rating_box = pd.DataFrame(dict)

In [14]:
len(corp)

2352

In [15]:
for i in corp :
    try:
        driver.get('https://www.jobplanet.co.kr/search?query='+i+'&category=&_rs_con=search&_rs_act=index&_rs_element=main_search_bar')
        icon = driver.find_element_by_class_name("tit")
        icon.click()
        time.sleep(5)

        # 페이지소스 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html)
        # 복지 및 급여
        welfare = soup.find_all(class_='txt_point')[0].text
        # 업무와 삶의 균형
        balance = soup.find_all(class_='txt_point')[1].text
        # 사내문화
        culture = soup.find_all(class_='txt_point')[2].text
        # 승진 기회 및 가능성
        possibility = soup.find_all(class_='txt_point')[3].text
        # 경영진
        executive = soup.find_all(class_='txt_point')[4].text
        # 총 만족도
        total = soup.find_all(class_='rate_point')[0].text
        name = soup.find_all(class_='word')[0].text
        dict = {'회사명':[i],'네임':[name],'복지':[welfare],'워라벨':[balance],'사내문화':[culture],'가능성':[possibility],'경영진':[executive],'총만족도':[total]}
        rating_box = rating_box.append(pd.DataFrame(dict))
    
    except: continue

In [16]:
rating_box

,회사명,네임,복지,워라벨,사내문화,가능성,경영진,총만족도
0,GS,GS,3.9,3.3,3.4,3.3,2.8,3.5
0,DSR,DSR,3.1,2.1,2.3,2.5,2.2,2.6
0,GS,GS,3.9,3.3,3.4,3.3,2.8,3.5
0,GS글로벌,GS글로벌,3.1,2.4,2.1,2.7,2.3,2.7
0,HDC현대산업개발,HDC현대산업개발,3.5,3.2,3.2,3.1,2.8,3.5
...,...,...,...,...,...,...,...,...
0,이노진,이노진,1.8,2.7,2.4,2.3,1.5,2.0
0,코셋,코셋,2.3,2.3,3.3,4.0,3.0,3.3
0,툴젠,툴젠,2.6,3.3,2.7,2.4,2.2,2.6
0,한국미라클피플사,한국미라클피플사,2.0,3.9,2.0,1.7,1.6,1.7


## rating_box.csv 로 원본파일 저장

In [17]:
rating_box.to_csv('rating_box.csv')

In [22]:
len(rating_box)

2127

In [38]:
null_box1=[]
for i in range(len(rating_box)) :
    if rating_box.iloc[i,:]['회사명']!=rating_box.iloc[i,:]['네임'] :
        print(rating_box.iloc[i,:]['회사명'])
        null_box1.append(rating_box.iloc[i,:]['회사명'])

S&TC
S&T모티브
THE E&M
세이브존I&C
SM C&C
형지I&C
F&F
S&T중공업
S&T홀딩스
동원F&B
신세계I&C
SG&G
서부T&D
무림P&P
동국S&C
S&K폴리텍
백금T&A


In [36]:
null_box2=[]
for i in range(len(rating_box)) :
    if rating_box.iloc[i,:]['복지']=='-' :
        print(rating_box.iloc[i,:]['회사명'])
        null_box2.append(rating_box.iloc[i,:]['회사명'])

GH신소재
삼본전자
에스아이리소스
엠벤처투자
제일제강
카이노스메드
신일전자
쿠쿠홀딩스
에스제이케이
윈하이텍
코너스톤네트웍스
MH에탄올
농심홀딩스
한국주강
한솔홀딩스
데일리블록체인
이건홀딩스
헬릭스미스
두산솔루스
영원무역홀딩스
한진중공업홀딩스
스튜디오드래곤
에이비프로바이오
컴퍼니케이
피에스케이홀딩스
KISCO홀딩스
SV인베스트먼트
골드퍼시픽
골든센츄리
노바텍
동양파일
로스웰
메드팩토
이큐셀
큐리언트
명진홀딩스
에스제이엠홀딩스
일동홀딩스
대림제지
위니아딤채
지노믹트리
코리아에스이
넥스트사이언스
티웨이홀딩스
나우IB
버추얼텍
우리기술투자
더콘텐츠온
안지오랩
에스엔피제네틱스
대우부품
진양홀딩스
대보마그네틱
매커스
에이치엘비생명과학
중앙오션


In [41]:
null_box1

['S&TC',
 'S&T모티브',
 'THE E&M',
 '세이브존I&C',
 'SM C&C',
 '형지I&C',
 'F&F',
 'S&T중공업',
 'S&T홀딩스',
 '동원F&B',
 '신세계I&C',
 'SG&G',
 '서부T&D',
 '무림P&P',
 '동국S&C',
 'S&K폴리텍',
 '백금T&A']

In [44]:
# null값 제거하기
rating_box1 = rating_box.loc[rating_box['복지']!='-',:]

rating_box1.index=rating_box1['회사명']

last_rating = rating_box1.drop(null_box1,axis=0)

last_rating = last_rating.drop(['회사명','네임'],axis=1)

In [56]:
last_rating

,복지,워라벨,사내문화,가능성,경영진,총만족도
회사명,,,,,,
GS,3.9,3.3,3.4,3.3,2.8,3.5
DSR,3.1,2.1,2.3,2.5,2.2,2.6
GS,3.9,3.3,3.4,3.3,2.8,3.5
GS글로벌,3.1,2.4,2.1,2.7,2.3,2.7
HDC현대산업개발,3.5,3.2,3.2,3.1,2.8,3.5
...,...,...,...,...,...,...
이노진,1.8,2.7,2.4,2.3,1.5,2.0
코셋,2.3,2.3,3.3,4.0,3.0,3.3
툴젠,2.6,3.3,2.7,2.4,2.2,2.6


In [59]:
# merging 위해 reset_index
last_rating = last_rating.reset_index()

In [60]:
last_rating

,회사명,복지,워라벨,사내문화,가능성,경영진,총만족도
0,GS,3.9,3.3,3.4,3.3,2.8,3.5
1,DSR,3.1,2.1,2.3,2.5,2.2,2.6
2,GS,3.9,3.3,3.4,3.3,2.8,3.5
3,GS글로벌,3.1,2.4,2.1,2.7,2.3,2.7
4,HDC현대산업개발,3.5,3.2,3.2,3.1,2.8,3.5
...,...,...,...,...,...,...,...
2049,이노진,1.8,2.7,2.4,2.3,1.5,2.0
2050,코셋,2.3,2.3,3.3,4.0,3.0,3.3
2051,툴젠,2.6,3.3,2.7,2.4,2.2,2.6
2052,한국미라클피플사,2.0,3.9,2.0,1.7,1.6,1.7


In [64]:
corp_list

,회사명,종목코드,업종,지역
0,DSR,155660,1차 비철금속 제조업,부산광역시
1,GS,078930,기타 금융업,서울특별시
2,GS글로벌,001250,상품 종합 도매업,서울특별시
3,HDC현대산업개발,294870,건물 건설업,서울특별시
4,LG이노텍,011070,전자부품 제조업,서울특별시
...,...,...,...,...
2347,지앤이헬스케어,299480,기타 섬유제품 제조업,경기도
2348,코셋,189350,통신 및 방송 장비 제조업,광주광역시
2349,툴젠,199800,자연과학 및 공학 연구개발업,서울특별시
2350,한국미라클피플사,331660,기타 화학제품 제조업,경기도


In [65]:
table1 = pd.merge(corp_list,last_rating, how="inner", on='회사명')

In [66]:
table1

,회사명,종목코드,업종,지역,복지,워라벨,사내문화,가능성,경영진,총만족도
0,DSR,155660,1차 비철금속 제조업,부산광역시,3.1,2.1,2.3,2.5,2.2,2.6
1,GS,078930,기타 금융업,서울특별시,3.9,3.3,3.4,3.3,2.8,3.5
2,GS,078930,기타 금융업,서울특별시,3.9,3.3,3.4,3.3,2.8,3.5
3,GS글로벌,001250,상품 종합 도매업,서울특별시,3.1,2.4,2.1,2.7,2.3,2.7
4,HDC현대산업개발,294870,건물 건설업,서울특별시,3.5,3.2,3.2,3.1,2.8,3.5
...,...,...,...,...,...,...,...,...,...,...
2049,이노진,344860,기타 화학제품 제조업,서울특별시,1.8,2.7,2.4,2.3,1.5,2.0
2050,코셋,189350,통신 및 방송 장비 제조업,광주광역시,2.3,2.3,3.3,4.0,3.0,3.3
2051,툴젠,199800,자연과학 및 공학 연구개발업,서울특별시,2.6,3.3,2.7,2.4,2.2,2.6
2052,한국미라클피플사,331660,기타 화학제품 제조업,경기도,2.0,3.9,2.0,1.7,1.6,1.7


In [69]:
len(table1['업종'].value_counts())

152

In [73]:
list(table1['업종'].value_counts())

[126,
 117,
 114,
 89,
 87,
 80,
 68,
 54,
 53,
 52,
 44,
 42,
 41,
 40,
 36,
 36,
 33,
 32,
 30,
 29,
 28,
 24,
 24,
 23,
 22,
 21,
 21,
 20,
 19,
 17,
 17,
 17,
 15,
 15,
 14,
 13,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [80]:
업종 = table1['업종'].value_counts().reset_index()
업종

,index,업종
0,특수 목적용 기계 제조업,126
1,소프트웨어 개발 및 공급업,117
2,전자부품 제조업,114
3,자동차 신품 부품 제조업,89
4,의약품 제조업,87
...,...,...
147,운송장비 임대업,1
148,그외 기타 개인 서비스업,1
149,환경 정화 및 복원업,1
150,재 보험업,1


In [81]:
업종.columns = ['업종','개수']

In [82]:
업종

,업종,개수
0,특수 목적용 기계 제조업,126
1,소프트웨어 개발 및 공급업,117
2,전자부품 제조업,114
3,자동차 신품 부품 제조업,89
4,의약품 제조업,87
...,...,...
147,운송장비 임대업,1
148,그외 기타 개인 서비스업,1
149,환경 정화 및 복원업,1
150,재 보험업,1


In [83]:
업종[업종['개수']>50]

,업종,개수
0,특수 목적용 기계 제조업,126
1,소프트웨어 개발 및 공급업,117
2,전자부품 제조업,114
3,자동차 신품 부품 제조업,89
4,의약품 제조업,87
5,기타 화학제품 제조업,80
6,통신 및 방송 장비 제조업,68
7,기타 금융업,54
8,반도체 제조업,53
9,1차 철강 제조업,52


In [84]:
업종[업종['개수']>30]

,업종,개수
0,특수 목적용 기계 제조업,126
1,소프트웨어 개발 및 공급업,117
2,전자부품 제조업,114
3,자동차 신품 부품 제조업,89
4,의약품 제조업,87
5,기타 화학제품 제조업,80
6,통신 및 방송 장비 제조업,68
7,기타 금융업,54
8,반도체 제조업,53
9,1차 철강 제조업,52


In [86]:
업종[업종['개수']<20]

,업종,개수
28,토목 건설업,19
29,"구조용 금속제품, 탱크 및 증기발생기 제조업",17
30,"자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업",17
31,1차 비철금속 제조업,17
32,영상 및 음향기기 제조업,15
...,...,...
147,운송장비 임대업,1
148,그외 기타 개인 서비스업,1
149,환경 정화 및 복원업,1
150,재 보험업,1


In [90]:
업종

,업종,개수
0,특수 목적용 기계 제조업,126
1,소프트웨어 개발 및 공급업,117
2,전자부품 제조업,114
3,자동차 신품 부품 제조업,89
4,의약품 제조업,87
...,...,...
147,운송장비 임대업,1
148,그외 기타 개인 서비스업,1
149,환경 정화 및 복원업,1
150,재 보험업,1


In [89]:
업종['종류']=[업종[업종['개수']<10]]['개수']

,업종,개수
47,상품 중개업,9
48,화학섬유 제조업,9
49,기타 비금속 광물제품 제조업,9
50,"서적, 잡지 및 기타 인쇄물 출판업",9
51,"골판지, 종이 상자 및 종이용기 제조업",9
...,...,...
147,운송장비 임대업,1
148,그외 기타 개인 서비스업,1
149,환경 정화 및 복원업,1
150,재 보험업,1


In [92]:
len(업종)

152

In [112]:
업종

,업종,개수,종류
0,특수 목적용 기계 제조업,126,광어업
1,소프트웨어 개발 및 공급업,117,광어업
2,전자부품 제조업,114,광어업
3,자동차 신품 부품 제조업,89,광어업
4,의약품 제조업,87,광어업
...,...,...,...
147,운송장비 임대업,1,광어업
148,그외 기타 개인 서비스업,1,광어업
149,환경 정화 및 복원업,1,광어업
150,재 보험업,1,광어업


In [175]:
for i in range(len(업종)) :
    if '제조업' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '제조업'
        
    elif '소매업' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '소매업'
        
    elif '임대' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '임대업'
        
    elif '건설업' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '건설업'
    elif '공사업' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '건설업'
        
    elif '서비스' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '서비스'
    elif '광고' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '서비스'
    elif '경비' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '서비스'
    elif '경영' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '서비스'
    elif '음식점' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '서비스'
        
    elif '도매' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '도매업'
    elif '처리' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '도매업'
        
    elif '교육' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '교육업'
    elif '학원' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '교육업'    
    elif '금속' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '금속업'
        
    elif '금융' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '금융'
    elif '은행' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '금융'
    elif '보험' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '금융'
    elif '중개' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '금융'
    elif '투자' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '금융'    
    elif '운송' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '운송업'
    elif '소프트웨어' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = 'it'
    elif '프로그래밍' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = 'it'
    elif '연구' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '연구'
    elif '영화' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '방송'
    elif '방송' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '방송' 
    elif '선박' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '선박'    
    elif '인쇄' in 업종.loc[i,'업종'] :
        업종.loc[i,'종류'] = '인쇄'  
    else : 업종.loc[i,'종류'] = '기타'

In [177]:
업종

,업종,개수,종류
0,특수 목적용 기계 제조업,126,제조업
1,소프트웨어 개발 및 공급업,117,it
2,전자부품 제조업,114,제조업
3,자동차 신품 부품 제조업,89,제조업
4,의약품 제조업,87,제조업
...,...,...,...
147,운송장비 임대업,1,임대업
148,그외 기타 개인 서비스업,1,서비스
149,환경 정화 및 복원업,1,기타
150,재 보험업,1,금융


In [176]:
업종['종류'].value_counts()

제조업    66
서비스    21
기타     16
도매업    11
건설업     6
소매업     6
금융      6
운송업     4
임대업     4
교육업     3
인쇄      2
it      2
방송      2
연구      1
선박      1
금속업     1
Name: 종류, dtype: int64

In [183]:
table1 = pd.merge(table1, 업종, how='inner',on='업종')

In [178]:
mapping={"제조업": 0, "서비스" : 1, "도매업" : 2,
               "소매업" : 3, "건설업" : 4, "금융" : 5,
               "운송업" : 6, "임대업" : 7, "교육업" : 8,
               "인쇄" : 9, "it" : 10, "방송" : 11,
               "연구" : 12, "선박" : 13,
               "금속업" : 14, "기타" : 15}

In [186]:
table1['업종1']=table1['종류'].map(mapping)

In [187]:
table1

,회사명,종목코드,업종,지역,복지,워라벨,사내문화,가능성,경영진,총만족도,업종1,개수,종류
0,DSR,155660,1차 비철금속 제조업,부산광역시,3.1,2.1,2.3,2.5,2.2,2.6,0,17,제조업
1,고려아연,010130,1차 비철금속 제조업,서울특별시,3.1,3.2,2.6,3.1,2.7,3.3,0,17,제조업
2,삼보산업,009620,1차 비철금속 제조업,경상남도,1.5,1.2,1.5,1.7,1.3,1.6,0,17,제조업
3,남선알미늄,008350,1차 비철금속 제조업,대구광역시,2.8,2.9,2.7,3.1,2.1,2.7,0,17,제조업
4,이구산업,025820,1차 비철금속 제조업,경기도,1.5,1.9,1.8,2.4,1.7,1.8,0,17,제조업
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2049,볼빅,206950,운동 및 경기용구 제조업,충청북도,2.6,1.5,2.5,2.8,2.0,2.3,0,1,제조업
2050,케이티앤지,033780,담배 제조업,대전광역시,4.2,3.7,3.4,3.2,3.1,3.9,0,1,제조업
2051,한국전력공사,015760,전기업,전라남도,3.9,4.1,3.4,3.6,3.3,3.9,15,1,기타
2052,현대로템,064350,철도장비 제조업,경상남도,3.6,2.8,2.6,2.7,2.4,3.0,0,1,제조업


In [71]:
table1['지역'].value_counts()

서울특별시      788
경기도        590
충청남도        84
경상남도        81
인천광역시       78
충청북도        73
부산광역시       72
경상북도        56
대구광역시       53
대전광역시       46
전라북도        24
강원도         24
울산광역시       24
광주광역시       21
전라남도        19
세종특별자치시      8
제주특별자치도      6
일본           3
미국           2
홍콩           1
케이맨 제도       1
Name: 지역, dtype: int64

In [190]:
mapping={"서울특별시": 0, "경기도" : 1, "충청남도" : 2,
               "경상남도" : 3, "인천광역시" : 4, "충청북도" : 5,
               "부산광역시" : 6, "경상북도" : 7, "대구광역시" : 8,
               "대전광역시" : 9, "전라북도" : 10, "강원도" : 11,
               "울산광역시" : 12, "광주광역시" : 13,
               "전라남도" : 14, "세종특별자치시" : 15,
            "제주특별자치도" : 16, "일본" : 17,
            "미국" : 18, "홍콩" : 19,"케이맨 제도" : 20,}

In [191]:
table1['지역1']=table1['지역'].map(mapping)

In [199]:
table1 = table1.drop(['종목코드','업종','지역','종류'],axis=1)

In [201]:
table1.columns = ['회사명', '복지', '워라벨', '사내문화', '가능성', '경영진', '총만족도', '업종분류', '업종개수', '지역분류']

In [202]:
table1

,회사명,복지,워라벨,사내문화,가능성,경영진,총만족도,업종분류,업종개수,지역분류
0,DSR,3.1,2.1,2.3,2.5,2.2,2.6,0,17,6
1,고려아연,3.1,3.2,2.6,3.1,2.7,3.3,0,17,0
2,삼보산업,1.5,1.2,1.5,1.7,1.3,1.6,0,17,3
3,남선알미늄,2.8,2.9,2.7,3.1,2.1,2.7,0,17,8
4,이구산업,1.5,1.9,1.8,2.4,1.7,1.8,0,17,1
...,...,...,...,...,...,...,...,...,...,...
2049,볼빅,2.6,1.5,2.5,2.8,2.0,2.3,0,1,5
2050,케이티앤지,4.2,3.7,3.4,3.2,3.1,3.9,0,1,9
2051,한국전력공사,3.9,4.1,3.4,3.6,3.3,3.9,15,1,14
2052,현대로템,3.6,2.8,2.6,2.7,2.4,3.0,0,1,3


In [203]:
table1.to_csv('table1.csv')